<h1>From Data Camp: Introduction to Databases in Python</h1>

<h3>Chapter 5: Putting it all together</h3>

* Preparing SQLAlchemy and the database
* Loading data into the database
* Solving data science problems with queries

The case study is broken down into three parts. First, I'm going to prepare SQLAlchemy and the database. Next, I will load the data into the database. And finally, we solve a few data science type problems with my query knowledge.



<b>Preparing SQLAlchemy and the database</b>

In [59]:
# Import Table, Column, String, Integer, Float, Boolean from sqlalchemy
from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer, insert, select, func, case, cast
from sqlalchemy import Float, desc

# import pandas
import pandas as pd

# import pandas
import csv

In [34]:
# Define an engine to connect to chapter5.sqlite: engine
engine = create_engine('sqlite:///chapter5.sqlite')

# Initialize MetaData: metadata
metadata = MetaData()

# Create a connection on engine
connection = engine.connect()

In [4]:
# Build a census table: census
census = Table('census', metadata,
               Column('state', String(30)),
               Column('sex', String(1)),
               Column('age', Integer),
               Column('pop2000', Integer),
               Column('pop2008', Integer))

# Create the table in the database
metadata.create_all(engine)

<b>Populating the database</b>

In [29]:
# Creating a "census_csv_data" variable with CSV file directory
census_csv_data = ('0_Data/Census.csv')

# Create an empty list: values_list
values_list = list()

# Reading CSV file
with open(census_csv_data, newline='\n') as csvfile:
    csv_reader = csv.reader(csvfile)
    
  # Iterate over the rows
    for row in csv_reader:
        
        # Create a dictionary with the values
        data = {'state': row[0],
                'sex': row[1],
                'age':row[2],
                'pop2000': row[3],
                'pop2008': row[4]}
        
        # Append the dictionary to the values list
        values_list.append(data)

In [30]:
# Checking the dictionary list
values_list

[{'state': 'Illinois',
  'sex': 'M',
  'age': '0',
  'pop2000': '89600',
  'pop2008': '95012'},
 {'state': 'Illinois',
  'sex': 'M',
  'age': '1',
  'pop2000': '88445',
  'pop2008': '91829'},
 {'state': 'Illinois',
  'sex': 'M',
  'age': '2',
  'pop2000': '88729',
  'pop2008': '89547'},
 {'state': 'Illinois',
  'sex': 'M',
  'age': '3',
  'pop2000': '88868',
  'pop2008': '90037'},
 {'state': 'Illinois',
  'sex': 'M',
  'age': '4',
  'pop2000': '91947',
  'pop2008': '91111'},
 {'state': 'Illinois',
  'sex': 'M',
  'age': '5',
  'pop2000': '93894',
  'pop2008': '89802'},
 {'state': 'Illinois',
  'sex': 'M',
  'age': '6',
  'pop2000': '93676',
  'pop2008': '88931'},
 {'state': 'Illinois',
  'sex': 'M',
  'age': '7',
  'pop2000': '94818',
  'pop2008': '90940'},
 {'state': 'Illinois',
  'sex': 'M',
  'age': '8',
  'pop2000': '95035',
  'pop2008': '86943'},
 {'state': 'Illinois',
  'sex': 'M',
  'age': '9',
  'pop2000': '96436',
  'pop2008': '86055'},
 {'state': 'Illinois',
  'sex': 'M',
  '

In [42]:
len(values_list)

8772

In [35]:
# Build insert statement: stmt
stmt = insert(census)

# Use values_list to insert data: results
results = connection.execute(stmt, values_list)

# Print rowcount
print(results.rowcount)

8772


<b>Querying the database</b>

In [57]:
# Select the average of age weighted by pop2000
stmt = select([census.columns.sex, func.sum(census.columns.age * census.columns.pop2000) / 
               func.sum(census.columns.pop2000).label('average_age')])

# Group by sex
stmt = stmt.group_by(census.columns.sex)

# Execute the query and fetch all the results
results = connection.execute(stmt).fetchall()

# Print the sex and average age column for each result
for result in results:
    print(result[0], result[1])

F 37
M 34


In [54]:
# Build a query to calculate the percentage of women in 2000: stmt
stmt = select([census.columns.state,
    (func.sum(case([(census.columns.sex == 'F', census.columns.pop2000)], else_=0)) /
     cast(func.sum(census.columns.pop2000), Float) * 100).label('percent_female')])

# Group By state
stmt = stmt.group_by(census.columns.state)

# Execute the query and store the results: results
results = connection.execute(stmt).fetchall()

# Print the percentage
for result in results:
    print(result.state, result.percent_female)

Alabama 51.832407770179465
Alaska 49.301497893484594
Arizona 50.22361303057914
Arkansas 51.26992846221834
California 50.35233214901979
Colorado 49.84767060299562
Connecticut 51.66816507130644
Delaware 51.61109733558627
District of Columbia 53.129626141738385
Florida 51.36488001165242
Georgia 51.11408350339436
Hawaii 51.118011836915514
Idaho 49.98972623903102
Illinois 51.11224234802867
Indiana 50.95480313297678
Iowa 50.950398342534264
Kansas 50.821864107754735
Kentucky 51.32687036927168
Louisiana 51.75351596554121
Maine 51.50570813418951
Maryland 51.93575549972231
Massachusetts 51.843023571316785
Michigan 50.97246518318712
Minnesota 50.49332944301148
Mississippi 51.92229481794672
Missouri 51.46888602639692
Montana 50.32202690728538
Nebraska 50.8584549336086
Nevada 49.36736361384359
New Hampshire 50.858019844961746
New Jersey 51.51713956125773
New Mexico 51.0471720798335
New York 51.83453865150073
North Carolina 51.482262322084594
North Dakota 50.50069363231332
Ohio 51.46550350015544
Okl

In [60]:
# Build query to return state name and population difference from 2008 to 2000
stmt = select([census.columns.state,
     (census.columns.pop2008-census.columns.pop2000).label('pop_change')])

# Group by State
stmt = stmt.group_by(census.columns.state)

# Order by Population Change
stmt = stmt.order_by(desc('pop_change'))

# Limit to top 10
stmt = stmt.limit(10)

# Use connection to execute the statement and fetch all results
results = connection.execute(stmt).fetchall()

# Print the state and population change for each record
for result in results:
    print('{}:{}'.format(result.state, result.pop_change))

Texas:40137
California:35406
Florida:21954
Arizona:14377
Georgia:13357
North Carolina:11574
Virginia:6639
Colorado:6425
Utah:5934
Illinois:5412
